In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import urllib,time
import tushare as ts

# 1. 极端值涨跌幅预测因子

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 14:31:49 2019

@author: hurenjie
"""

# 极端值因子：当大盘(或者其他股票)日涨幅/跌幅绝对值超过2%，下一天上涨或者下跌的概率/历史上的次数有多少
#%%

token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
df = ts.get_hist_data('sh').sort_index()

#%%
def extreme_value(df, threshold): # threshold 在此应该定义为一个百分数,代表我们认为的涨跌幅度阈值
    df['close_pct_ch'] = df['close'].pct_change().shift(-1)
    df['close_diff'] = df['close'].diff().shift(-2)
    df_up = df[df['close_diff'] > 0]
    df_down = df[df['close_diff'] < 0]
    
    up = df_up[df_up['close_pct_ch'] > threshold]
    down = df_down[df_down['close_pct_ch'] < -threshold]
    
    prob_up = np.shape(up)[0]/np.shape(df_up)[0]
    prob_down = np.shape(down)[0]/np.shape(df_down)[0]
        
    if df['close_diff'].iloc[-3] > 0:
        result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
        print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
    else:
        result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
        print('当天价格下跌,该函数计算的是第二天价格继续下跌的概率')
    df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)
    return result

timed out


In [5]:
extreme_value(df, 0.0032)

当天价格下跌,该函数计算的是第二天价格继续下跌的概率


0.3082191780821918

In [6]:
# threshold = 0.0156
# df['close_pct_ch'] = df['close'].pct_change().shift(-1)
# df['close_diff'] = df['close'].diff().shift(-2)
# df_up = df[df['close_diff'] > 0]
# df_down = df[df['close_diff'] < 0]

# up = df_up[df_up['close_pct_ch'] > threshold]
# down = df_down[df_down['close_pct_ch'] < -threshold]

# prob_up = np.shape(up)[0]/np.shape(df_up)[0]
# prob_down = np.shape(down)[0]/np.shape(df_down)[0]

# if df['close_diff'].iloc[-3] > 0:
#     result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
#     print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
# else:
#     result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
#     print('当天价格下跌,该函数计算的事第二天价格继续下跌的概率')
# # df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)

# 2. 简单连续涨跌预测因子

In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 16:15:22 2019

@author: hurenjie
"""
# 简单连续涨跌因子：连续涨/跌N（3、4、5、6）天之后，下一天涨/跌的概率
#%% 
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
# import tushare as ts

#token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
#pro = ts.pro_api(token)
#df = ts.get_hist_data('sh').sort_index()

#%%
def simple_predict_up_or_down(df, N): # N代表我们认为的连续多少天持续的涨或者跌
    df.reset_index(inplace = True)
    df['close_diff'] = df['close'].diff()
    df['signal'] = 0
    for i in df.index:
        if df['close_diff'][i] < 0:
            df['signal'][i] = -1
        elif df['close_diff'][i] > 0:
            df['signal'][i] = 1
    
    df['signal_sum'] = df['signal'].rolling(N).sum()
    df_up_Ndays = df[df['signal_sum'] == N]
    next_day_is_up = 0
    
    df_down_Ndays = df[df['signal_sum'] == -N]
    next_day_is_down = 0
    
    for i in df_up_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] > 0:
                next_day_is_up += 1
    
    for i in df_down_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] < 0:
                next_day_is_down += 1
    df.drop(columns = ['close_diff', 'signal', 'signal_sum'], inplace = True)
    df.set_index('date', inplace = True)
    
    prob_up = next_day_is_up/np.shape(df_up_Ndays)[0]
    prob_down = next_day_is_down/np.shape(df_down_Ndays)[0]
    
    return {'连续'+ str(N) +'天上涨后第二天上涨的概率': prob_up, '连续'+ str(N) +'天下跌后第二天下跌的概率':prob_down}

In [20]:
simple_predict_up_or_down(df, 4)

C:\Users\bradsun91\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bradsun91\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'连续4天上涨后第二天上涨的概率': 0.5, '连续4天下跌后第二天下跌的概率': 0.3235294117647059}

# 3. 乖离率反弹因子

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug  7 10:26:08 2019

@author: hurenjie
"""
# 乖离率绝对值太大时，或者大到什么幅度时，大概率回归？
#%%
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import talib

token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
df = ts.get_hist_data('sh').sort_index()
df.index = pd.to_datetime(df.index)
#%%
def bias_next_day_prediction(df, zscorethreshold, bias_time_window=30, zscore_time_window=30, return_date = False):
    df['close_diff'] = df['close'].diff().shift(-1)
    
#    compute bias
    ave = df['close'].rolling(bias_time_window).mean()
    bias = (df['close']-ave)/ave
    
#    compute zscore
    df['bias'] = bias
    df['zscore'] = (df['bias'] - df['bias'].rolling(zscore_time_window).mean())/df['bias'].rolling(zscore_time_window).apply(np.std)
    
    larger_than_positive_threshold = df[df['zscore'] >= zscorethreshold]
    smaller_than_negative_threshold = df[df['zscore'] <= zscorethreshold]
    
    larger_than_positive_threshold_nextday_down = larger_than_positive_threshold[larger_than_positive_threshold['close_diff']<0]
    smaller_than_negative_threshold_nextday_up = smaller_than_negative_threshold[smaller_than_negative_threshold['close_diff']>0]
    
    if (df.index[-1] not in larger_than_positive_threshold.index) & (df.index[-1] not in smaller_than_negative_threshold.index):
        print(str(df.index[-1].date()) + '并没有发生z-score超出阈值'+str(zscorethreshold)+'的现象')
    elif (df.index[-1] in larger_than_positive_threshold.index):
        print(str(df.index[-1].date()) + '向上超出了z-score设定的阈值'+str(zscorethreshold))
    elif (df.index[-1] in smaller_than_negative_threshold.index):
        print(str(df.index[-1].date()) + '向下超出了z-score设定的阈值'+str(zscorethreshold))
    
    try:
        larger_than_positive_threshold_nextday_down_prob = larger_than_positive_threshold_nextday_down.shape[0]/larger_than_positive_threshold.shape[0]
        smaller_than_negative_threshold_nextday_up_prob = smaller_than_negative_threshold_nextday_up.shape[0]/smaller_than_negative_threshold.shape[0]
#        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
    except:
        print('阈值选取不合适，请选取小一点的阈值 zscorethreshold')
        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
    else:
        df.drop(columns = ['close_diff', 'bias', 'zscore'], inplace =  True)
        if return_date == False:
            return {'输入的数据中乖离率zscore大于'+str(zscorethreshold)+'时第二天下跌的概率':larger_than_positive_threshold_nextday_down_prob,
                    '输入的数据中乖离率zscore小于'+str(zscorethreshold)+'时第二天上涨的概率':smaller_than_negative_threshold_nextday_up_prob}
        elif return_date == True :
            return {'输入的数据中乖离率zscore大于'+str(zscorethreshold)+'时第二天下跌的概率':larger_than_positive_threshold_nextday_down_prob,
                    '输入的数据中乖离率zscore小于'+str(zscorethreshold)+'时第二天上涨的概率':smaller_than_negative_threshold_nextday_up_prob,
                    '向上超出zscore阈值'+str(zscorethreshold)+'的时间点':list(larger_than_positive_threshold.index),
                    '向下超出zscore阈值'+str(zscorethreshold)+'的时间点':list(smaller_than_negative_threshold.index)}
    

#%%
bias_next_day_prediction(df, zscorethreshold=2, return_date = True)

ModuleNotFoundError: No module named 'talib'

# 网易爬数据

In [25]:
def get_page(url):  #获取页面数据
    req=urllib.request.Request(url,headers={
        'Connection': 'Keep-Alive',
        'Accept': 'text/html, application/xhtml+xml, */*',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko'
    })
    opener=urllib.request.urlopen(req)
    page=opener.read()
    return page

def get_index_history_byNetease(index_temp):
    """
    :param index_temp: for example, 'sh000001' 上证指数
    :return:
    """
    index_type=index_temp[0:2]
    index_id=index_temp[2:]
    if index_type=='sh':
        index_id='0'+index_id
    if index_type=="sz":
        index_id='1'+index_id
    url='http://quotes.money.163.com/service/chddata.html?code=%s&start=19900101&end=%s&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER;VATURNOVER'%(index_id,time.strftime("%Y%m%d"))

    page=get_page(url).decode('gb2312') #该段获取原始数据
    page=page.split('\r\n')
    col_info=page[0].split(',')   #各列的含义
    index_data=page[1:]     #真正的数据

    #为了与现有的数据库对应，这里我还修改了列名，大家不改也没关系
    col_info[col_info.index('日期')]='交易日期'   #该段更改列名称
    col_info[col_info.index('股票代码')]='指数代码'
    col_info[col_info.index('名称')]='指数名称'
    col_info[col_info.index('成交金额')]='成交额'

    index_data=[x.replace("'",'') for x in index_data]  #去掉指数编号前的“'”
    index_data=[x.split(',') for x in index_data]

    index_data=index_data[0:index_data.__len__()-1]   #最后一行为空，需要去掉
    pos1=col_info.index('涨跌幅')
    pos2=col_info.index('涨跌额')
    posclose=col_info.index('收盘价')
    index_data[index_data.__len__()-1][pos1]=0      #最下面行涨跌额和涨跌幅为None改为0
    index_data[index_data.__len__()-1][pos2]=0
    for i in range(0,index_data.__len__()-1):       #这两列中有些值莫名其妙为None 现在补全
        if index_data[i][pos2]=='None':
            index_data[i][pos2]=float(index_data[i][posclose])-float(index_data[i+1][posclose])
        if index_data[i][pos1]=='None':
            index_data[i][pos1]=(float(index_data[i][posclose])-float(index_data[i+1][posclose]))/float(index_data[i+1][posclose])

    # print(col_info)
    return [index_data,col_info]
# --------------------- 
# 版权声明：本文为CSDN博主「multiangle」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/u014595019/article/details/48445223

In [26]:
sh = get_index_history_byNetease("sh000001") 
df_ne = pd.DataFrame()
df_ne['columns_to_split'] = pd.DataFrame(sh).T[0]
col_list = list(pd.DataFrame(sh).T[1])[:12]
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: str(x))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("[", ""))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("]", ""))

In [90]:
df_ne = df_ne['columns_to_split'].str.split(",", 12, expand = True)
df_ne.columns = col_list
df_ne.sort_values("交易日期").tail()